# Assign NPIs!

In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from sodapy import Socrata

from npyi import npi

Get the 2016 open payments dataset

In [3]:
%%bigquery
SELECT
    source_year AS year,
    COUNT(is_male) AS birth_count
FROM `bigquery-public-data.samples.natality`
GROUP BY year
ORDER BY year DESC
LIMIT 15

,year,birth_count
0,2008,4255156
1,2007,4324008
2,2006,4273225
3,2005,4145619
4,2004,4118907
5,2003,4096092
6,2002,4027376
7,2001,4031531
8,2000,4063823
9,1999,3963465


In [2]:
%%bigquery df
SELECT 
    Covered_Recipient_Type, 
    Teaching_Hospital_Name, 
    Physician_First_name, 
    Physician_Last_Name,
    Recipient_Primary_Business_Street_Address_Line1, 
    Recipient_City
FROM `carbide-server-294318.op2016.OP2016`

Modify the columns slightly

In [3]:
df.columns = df.columns.str.lower()

# Insert NPI column
df['npi'] = np.NaN

In [26]:
df.head()

,covered_recipient_type,teaching_hospital_name,physician_first_name,physician_last_name,recipient_primary_business_street_address_line1,recipient_city,npi
0,Covered Recipient Physician,None,SAUNDERS,HUPP,601 PROVIDENCE PARK DR E,MOBILE,NaN
1,Covered Recipient Physician,None,GARLAN,LO,1850 S AZUSA AVE,HACIENDA HEIGHTS,NaN
2,Covered Recipient Physician,None,GENE,KIM,6400 FANNIN ST,HOUSTON,NaN
3,Covered Recipient Physician,None,RAJIV,LUTHRA,2440 M ST NW,WASHINGTON,NaN
4,Covered Recipient Physician,None,STEVEN,GOLDENBERG,48 HARRISON STREET,JOHNSON CITY,NaN


In [5]:
# NPI matching
def npi_match(df):
    # for loop to go through every row in the OP dataframe
    # Source 3
    for index, row in df.iterrows():

        # Is the provider individual or enterprise?
        if row.covered_recipient_type == 'Covered Recipient Teaching Hospital':
            # Source 2
            results = npi.search(search_params={'organization_name': row.teaching_hospital_name,
                                                'address_purpose': 'PRIMARY',
                                                'city': row.recipient_city})                       

        if row.covered_recipient_type == 'Covered Recipient Physician':
            # Source 2
            results = npi.search(search_params={'first_name': row.physician_first_name,
                                                'last_name': row.physician_last_name,
                                                'address_purpose': 'PRIMARY',
                                                'city': row.recipient_city})

        # Iterate through the list of matches to find the exact match
        matches = 0
        for j in range(results['result_count']):
            if results['results'][j]['addresses'][0]['address_1'] == row.recipient_primary_business_street_address_line1:
                # We have a match!
                matches += 1
            
            # End loop early if more than one match was found
            if matches >= 1:
                break

        # Check to see if we only found one match
        #if matches == 1:
            #row.npi = results['results'][0]['number']
            
    return df

In [8]:
df.iloc[0].covered_recipient_type

'Covered Recipient Physician'

In [31]:
# NPI matching2
def npi_match_1(df, start, stop):
    # for loop to go through every row in the OP dataframe
    # Source 3
    for i in range(start, stop+1):

        # Is the provider individual or enterprise?
        if df.iloc[i].covered_recipient_type == 'Covered Recipient Teaching Hospital':
            # Source 2
            results = npi.search(search_params={'organization_name': df.iloc[i].teaching_hospital_name,
                                                'address_purpose': 'PRIMARY',
                                                'city': df.iloc[i].recipient_city})                       

        if df.iloc[i].covered_recipient_type == 'Covered Recipient Physician':
            # Source 2
            results = npi.search(search_params={'first_name': df.iloc[i].physician_first_name,
                                                'last_name': df.iloc[i].physician_last_name,
                                                'address_purpose': 'PRIMARY',
                                                'city': df.iloc[i].recipient_city})

        # Iterate through the list of matches to find the exact match
        matches = 0
        for j in range(results['result_count']):
            if results['results'][j]['addresses'][0]['address_1'] == df.iloc[i].recipient_primary_business_street_address_line1:
                # We have a match!
                matches += 1
            
            # End loop early if more than one match was found
            if matches >= 1:
                break

        # Check to see if we only found one match
        if matches == 1:
            df.iloc[i, df.columns.get_loc('npi')] = results['results'][0]['number']
            
    return df

In [ ]:
df_1 = npi_match_1(df, 0, 1_000_000)

In [ ]:
df.isnull().sum()

In [ ]:
df_2 = npi_match_1(df,1_000_001, 2_000_000)
df_3 = npi_match_1(df,2_000_001, 3_000_000)

In [ ]:
step = 0
next_step = 1_000_000
done = False

while !done:
    # Check if we're less than 1_000_000 from end
    if i + 1_000_000 > len(df):
        next_step = len(df+1)

    for i in range(step, next_step):

    #Check if we are done
    if next_step == len(df+1):
        done = True
    else:
        #increment steps
        steps = next_step
        next_step += 1_000_000


In [6]:
# Organization example
response = npi.search(search_params={'organization_name': 'PRESBYTERIAN HOSPITAL',
                                     'city': 'ALBUQUERQUE'})
#print(response.keys()) # dict_keys(['result_count', 'results'])
response

{'result_count': 1,
 'results': [{'enumeration_type': 'NPI-2',
   'number': 1215913470,
   'last_updated_epoch': 1557420354,
   'created_epoch': 1135036800,
   'basic': {'organization_name': 'PRESBYTERIAN HEALTHCARE SERVICES',
    'organizational_subpart': 'NO',
    'enumeration_date': '2005-12-20',
    'last_updated': '2019-05-09',
    'status': 'A',
    'authorized_official_first_name': 'KIMBERLY',
    'authorized_official_last_name': 'POLAND',
    'authorized_official_telephone_number': '505-923-5355',
    'authorized_official_title_or_position': 'MANAGER',
    'name': 'PRESBYTERIAN HEALTHCARE SERVICES'},
   'other_names': [{'organization_name': 'PRESBYTERIAN HOSPITAL',
     'code': '3',
     'type': 'Doing Business As'}],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '1100 CENTRAL SE',
     'address_2': '',
     'city': 'ALBUQUERQUE',
     'state': 'NM',
     'postal_co

In [7]:
# Individual example
response = npi.search(search_params={'first_name': 'Andrew', 'last_name': 'Jackson'})
#print(response.keys()) # dict_keys(['result_count', 'results'])
response

{'result_count': 10,
 'results': [{'enumeration_type': 'NPI-1',
   'number': 1558864132,
   'last_updated_epoch': 1599751477,
   'created_epoch': 1520553600,
   'basic': {'first_name': 'ANDREA',
    'last_name': 'JACKSON',
    'credential': 'PA-C',
    'sole_proprietor': 'NO',
    'gender': 'F',
    'enumeration_date': '2018-03-09',
    'last_updated': '2020-09-10',
    'status': 'A',
    'name': 'JACKSON ANDREA',
    'certification_date': '2020-09-10'},
   'other_names': [{'code': '1',
     'type': 'Former Name',
     'last_name': 'HOOVER',
     'first_name': 'ANDREA'}],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '1163 COUNTRY CLUB RD',
     'address_2': '',
     'city': 'MONONGAHELA',
     'state': 'PA',
     'postal_code': '150631013',
     'telephone_number': '724-258-1000'},
    {'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'MA

In [8]:
df.isnull().sum()

covered_recipient_type                                    0
teaching_hospital_name                             11655341
physician_first_name                                  41825
physician_last_name                                   41825
recipient_primary_business_street_address_line1           0
recipient_city                                            0
recipient_state                                         598
recipient_state_1                                       598
npi                                                11697166
dtype: int64

In [ ]:
# Organization search
if df.Covered_Recipient_Type == 'Covered Recipient Teaching Hospital'
    # Source 2
    npi.search(search_params={'organization_name': df.Teaching_Hospital_Name,
                              'address_purpose': 'PRIMARY',
                              'city': df.recipient_city,
                              'state': df.recipient_state})

In [ ]:
# Individual search
if df.Covered_Recipient_Type == 'Covered Recipient Physician'
    # Source 2
    npi.search(search_params={'first_name': df.physician_first_name, 
                              'last_name': df.physcian_last_name,
                              'address_purpose': 'PRIMARY',
                              'city': df.recipient_city,
                              'state': df.recipient_state})

In [ ]:
# Search resulting list for exact match using the addresses
for i in ?['result_count']:
    if ?['results'][i]['addresses'][0]['address_1'] == df.recipient_primary_business_street_address_1
        # We have a match!
        df.NPI = ?['results'][i]['number']

In [ ]:
npi.VALID_SEARCH_PARAMS

Sources
1. https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/56014
2. https://npyi.readthedocs.io/en/latest/
3. https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-dataframe-in-pandas